# 02.2 - Kedro on Databricks, part 2

_Notes: This notebook is supposed to be run locally from VS Code, all with Databricks Connect_

## Kedro and Databricks Connect

**Databricks Connect** is a client library that allows you to run Spark code locally on your machine while connecting to a remote Databricks cluster for computation. It essentially lets you develop and execute Spark applications from your local IDE or notebook environment, but the actual processing happens on the Databricks cluster.

The **Databricks extension for Visual Studio Code** has several interesting features for connecting to Databricks from VS Code and perform actions sach us deploying and running Databricks Asset Bundles, manage clusters, and easily set up **Databricks Connect**.

Therefore, the two are the perfect companion for developing Kedro projects on VS Code, since you can develop on your IDE while using Databricks compute.

Follow the official documentation to

1. [Install the Databricks extension for VS Code](https://learn.microsoft.com/en-us/azure/databricks/dev-tools/vscode-ext/install)
2. [Configure the appropriate cluster](https://learn.microsoft.com/en-us/azure/databricks/dev-tools/vscode-ext/configure)
3. [Install Databricks Connect](https://learn.microsoft.com/en-us/azure/databricks/dev-tools/vscode-ext/databricks-connect)

_Note: `databricks-connect` provides its own `pyspark` top-level module, and [pip doesn't check for conflicting packages](https://github.com/pypa/pip/issues/4625), so make sure you don't have a [conflicting `pyspark` installation](https://learn.microsoft.com/en-us/azure/databricks/dev-tools/databricks-connect/python/troubleshooting#conflicting-pyspark-installations)!_

In [1]:
%%sh
uv pip install ./rocketfuel

/Users/juan_cano/.ipython/profile_default/startup/00-databricks-init-cdb6174794c74fbc909de1ce43bfe286.py:341: UserWarning: 
%%sh is not supported on Databricks. This notebook might fail when running on a Databricks cluster.
Consider using %sh instead.
  warnings.warn(
Using Python 3.12.9 environment at: /Users/juan_cano/Projects/QuantumBlackLabs/Kedro/kedro-databricks-bootcamp/.venv
Resolved 179 packages in 20.71s
   Building rocketfuel @ file:///Users/juan_cano/Projects/QuantumBlackLabs/Kedro/kedro-databricks-bootcamp/02_databricks/rocketfuel
      Built rocketfuel @ file:///Users/juan_cano/Projects/QuantumBlackLabs/Kedro/kedro-databricks-bootcamp/02_databricks/rocketfuel
Prepared 3 packages in 3.50s
Uninstalled 5 packages in 1.54s
Installed 32 packages in 4.23s
 + aiobotocore==2.22.0
 + aiofiles==24.1.0
 + aiohappyeyeballs==2.6.1
 + aiohttp==3.12.11
 + aioitertools==0.12.0
 + aiosignal==1.3.2
 - botocore==1.34.69
 + botocore==1.37.3
 + click-default-group==1.2.4
 + delta-spark==3.3.2

In [2]:
%load_ext kedro.ipython

[06/09/25 16:52:54] INFO     Using                                                                  ]8;id=773116;file:///Users/juan_cano/Projects/QuantumBlackLabs/Kedro/kedro-databricks-bootcamp/.venv/lib/python3.12/site-packages/kedro/framework/project/__init__.py\__init__.py]8;;\:]8;id=519104;file:///Users/juan_cano/Projects/QuantumBlackLabs/Kedro/kedro-databricks-bootcamp/.venv/lib/python3.12/site-packages/kedro/framework/project/__init__.py#270\270]8;;\
                             '/Users/juan_cano/Projects/QuantumBlackLabs/Kedro/kedro-databricks-boo                
                             tcamp/.venv/lib/python3.12/site-packages/kedro/framework/project/rich_                
                             logging.yml' as logging configuration.                                                

                    INFO     Registered line magic '%reload_kedro'                                   ]8;id=395329;file:///Users/juan_cano/Projects/QuantumBlackLabs/Kedro/kedro-databricks-bootcamp/.venv/lib/python3.12/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=711460;file:///Users/juan_cano/Projects/QuantumBlackLabs/Kedro/kedro-databricks-bootcamp/.venv/lib/python3.12/site-packages/kedro/ipython/__init__.py#61\61]8;;\

                    INFO     Registered line magic '%load_node'                                      ]8;id=588601;file:///Users/juan_cano/Projects/QuantumBlackLabs/Kedro/kedro-databricks-bootcamp/.venv/lib/python3.12/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=828331;file:///Users/juan_cano/Projects/QuantumBlackLabs/Kedro/kedro-databricks-bootcamp/.venv/lib/python3.12/site-packages/kedro/ipython/__init__.py#63\63]8;;\

                    WARNING  Kedro extension was registered but couldn't find a Kedro project. Make  ]8;id=199965;file:///Users/juan_cano/Projects/QuantumBlackLabs/Kedro/kedro-databricks-bootcamp/.venv/lib/python3.12/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=746463;file:///Users/juan_cano/Projects/QuantumBlackLabs/Kedro/kedro-databricks-bootcamp/.venv/lib/python3.12/site-packages/kedro/ipython/__init__.py#66\66]8;;\
                             sure you run '%reload_kedro <project_root>'.                                          

In [3]:
%reload_kedro rocketfuel

[06/09/25 16:53:06] WARNING  /Users/juan_cano/Projects/QuantumBlackLabs/Kedro/kedro-databricks-boot ]8;id=124591;file:///opt/homebrew/Cellar/python@3.12/3.12.9/Frameworks/Python.framework/Versions/3.12/lib/python3.12/warnings.py\warnings.py]8;;\:]8;id=938420;file:///opt/homebrew/Cellar/python@3.12/3.12.9/Frameworks/Python.framework/Versions/3.12/lib/python3.12/warnings.py#112\112]8;;\
                             camp/.venv/lib/python3.12/site-packages/mlflow/pyfunc/utils/data_valid                
                             ation.py:186: UserWarning: [33mAdd type hints to the `predict` method                 
                             to enable data validation and automatic signature inference during                    
                             model logging. Check                                                                  
                             https://mlflow.org/docs/latest/model/python_model.html#type-hint-usage                
                             -in-pythonmodel for more details.[0m                                                  
                               color_warning(                                                                      
                                                                                                                   

[06/09/25 16:53:07] INFO     Registering new custom resolver: 'km.random_name'                    ]8;id=442291;file:///Users/juan_cano/Projects/QuantumBlackLabs/Kedro/kedro-databricks-bootcamp/.venv/lib/python3.12/site-packages/kedro_mlflow/framework/hooks/mlflow_hook.py\mlflow_hook.py]8;;\:]8;id=804281;file:///Users/juan_cano/Projects/QuantumBlackLabs/Kedro/kedro-databricks-bootcamp/.venv/lib/python3.12/site-packages/kedro_mlflow/framework/hooks/mlflow_hook.py#65\65]8;;\

                    WARNING  No 'mlflow.yml' config file found in environment. Default            ]8;id=608380;file:///Users/juan_cano/Projects/QuantumBlackLabs/Kedro/kedro-databricks-bootcamp/.venv/lib/python3.12/site-packages/kedro_mlflow/framework/hooks/mlflow_hook.py\mlflow_hook.py]8;;\:]8;id=525126;file:///Users/juan_cano/Projects/QuantumBlackLabs/Kedro/kedro-databricks-bootcamp/.venv/lib/python3.12/site-packages/kedro_mlflow/framework/hooks/mlflow_hook.py#78\78]8;;\
                             configuration will be used. Use ``kedro mlflow init`` command in CLI                  
                             to customize the configuration.                                                       

                    INFO     The 'tracking_uri' key in mlflow.yml is relative            ]8;id=5410;file:///Users/juan_cano/Projects/QuantumBlackLabs/Kedro/kedro-databricks-bootcamp/.venv/lib/python3.12/site-packages/kedro_mlflow/config/kedro_mlflow_config.py\kedro_mlflow_config.py]8;;\:]8;id=129235;file:///Users/juan_cano/Projects/QuantumBlackLabs/Kedro/kedro-databricks-bootcamp/.venv/lib/python3.12/site-packages/kedro_mlflow/config/kedro_mlflow_config.py#293\293]8;;\
                             ('server.mlflow_(tracking|registry)_uri = mlruns'). It is                             
                             converted to a valid uri:                                                             
                             'file:///Users/juan_cano/Projects/QuantumBlackLabs/Kedro/ke                           
                             dro-databricks-bootcamp/02_databricks/rocketfuel/mlruns'                              

[06/09/25 16:53:19] INFO     Kedro is sending anonymous usage data with the sole purpose of improving ]8;id=301338;file:///Users/juan_cano/Projects/QuantumBlackLabs/Kedro/kedro-databricks-bootcamp/.venv/lib/python3.12/site-packages/kedro_telemetry/plugin.py\plugin.py]8;;\:]8;id=86750;file:///Users/juan_cano/Projects/QuantumBlackLabs/Kedro/kedro-databricks-bootcamp/.venv/lib/python3.12/site-packages/kedro_telemetry/plugin.py#233\233]8;;\
                             the product. No personal data or IP addresses are stored on our side. If              
                             you want to opt out, set the `KEDRO_DISABLE_TELEMETRY` or `DO_NOT_TRACK`              
                             environment variables, or create a `.telemetry` file in the current                   
                             working directory with the contents `consent: false`. Read more at                    
                             https://docs.kedro.org/en/stable/configuration/telemetry.html                         

[06/09/25 16:53:20] INFO     Kedro project rocketfuel                                               ]8;id=206079;file:///Users/juan_cano/Projects/QuantumBlackLabs/Kedro/kedro-databricks-bootcamp/.venv/lib/python3.12/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=861539;file:///Users/juan_cano/Projects/QuantumBlackLabs/Kedro/kedro-databricks-bootcamp/.venv/lib/python3.12/site-packages/kedro/ipython/__init__.py#144\144]8;;\

                    INFO     Defined global variable 'context', 'session', 'catalog' and            ]8;id=160147;file:///Users/juan_cano/Projects/QuantumBlackLabs/Kedro/kedro-databricks-bootcamp/.venv/lib/python3.12/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=612888;file:///Users/juan_cano/Projects/QuantumBlackLabs/Kedro/kedro-databricks-bootcamp/.venv/lib/python3.12/site-packages/kedro/ipython/__init__.py#145\145]8;;\
                             'pipelines'                                                                           

[06/09/25 16:53:22] INFO     Registered line magic 'run_viz'                                        ]8;id=685882;file:///Users/juan_cano/Projects/QuantumBlackLabs/Kedro/kedro-databricks-bootcamp/.venv/lib/python3.12/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=425788;file:///Users/juan_cano/Projects/QuantumBlackLabs/Kedro/kedro-databricks-bootcamp/.venv/lib/python3.12/site-packages/kedro/ipython/__init__.py#151\151]8;;\

In [4]:
catalog.list()


[
    'companies',
    'reviews',
    'shuttles',
    'preprocessed_companies',
    'preprocessed_shuttles',
    'preprocessed_reviews',
    'model_input_table',
    'regressor',
    'parameters',
    'params:model_options',
    'params:model_options.test_size',
    'params:model_options.random_state',
    'params:model_options.features'
]

Notice how data is loaded as a PySpark DataFrame, directly from Databricks Unity Catalog!

In [5]:
catalog._get_dataset("companies")

kedro_datasets.databricks.managed_table_dataset.ManagedTableDataset(catalog='aza-databricks-b9b7aae-catalog', database='rocketfuel', table='companies', write_mode='overwrite', dataframe_type='spark', version='None')

In [6]:
display(catalog.load("companies"))

[06/09/25 16:53:35] INFO     Loading data from companies (ManagedTableDataset)...               ]8;id=886188;file:///Users/juan_cano/Projects/QuantumBlackLabs/Kedro/kedro-databricks-bootcamp/.venv/lib/python3.12/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=593487;file:///Users/juan_cano/Projects/QuantumBlackLabs/Kedro/kedro-databricks-bootcamp/.venv/lib/python3.12/site-packages/kedro/io/data_catalog.py#389\389]8;;\

,id,company_rating,company_location,total_fleet_count,iata_approved
0,3888,100%,Isle of Man,1.0,f
1,46728,100%,None,1.0,f
2,34618,38%,Isle of Man,1.0,f
3,28619,100%,Bosnia and Herzegovina,1.0,f
4,8240,None,Chile,1.0,t
5,16813,100%,Kiribati,2.0,f
6,2859,90%,Bahrain,1.0,f
7,33237,None,Nicaragua,1.0,f
8,30052,100%,Turkmenistan,1.0,f
9,43711,100%,Rwanda,1.0,f


## Namespaces for pipeline grouping

## Integration with Databricks MLflow

### Log Kedro runs as MLflow experiments

There are 2 types of MLflow experiments in Databricks:
- **Workspace** experiments are not associated with any notebook, and any notebook can log a run to these experiments by using the experiment ID or the experiment name. _They cannot be created inside Git folders._
- **Notebook** experiments are associated with a specific notebook. _They are note checked into source control_.

Therefore, for personal experimentation **notebook** experiments are more appropriate, and for collaboration **workspace** experiments can be created in a regular workspace folder outside of Git.

In [0]:
%%sh
uv pip install ./rocketfuel

In [0]:
%load_ext kedro.ipython

Kedro runs can be logged in MLflow using the [`kedro-mlflow`](https://kedro-mlflow.readthedocs.io/) community plugin.

In [7]:
%%sh
uv pip install kedro-mlflow

[06/09/25 16:54:11] WARNING  /Users/juan_cano/.ipython/profile_default/startup/00-databricks-init-c ]8;id=523748;file:///opt/homebrew/Cellar/python@3.12/3.12.9/Frameworks/Python.framework/Versions/3.12/lib/python3.12/warnings.py\warnings.py]8;;\:]8;id=81443;file:///opt/homebrew/Cellar/python@3.12/3.12.9/Frameworks/Python.framework/Versions/3.12/lib/python3.12/warnings.py#112\112]8;;\
                             db6174794c74fbc909de1ce43bfe286.py:341: UserWarning:                                  
                             %%sh is not supported on Databricks. This notebook might fail when                    
                             running on a Databricks cluster.                                                      
                             Consider using %sh instead.                                                           
                               warnings.warn(                                                                      
                                                                                                                   

Using Python 3.12.9 environment at: /Users/juan_cano/Projects/QuantumBlackLabs/Kedro/kedro-databricks-bootcamp/.venv
Audited 1 package in 979ms


`kedro-mlflow` can take [configuration](https://kedro-mlflow.readthedocs.io/en/0.14.4/source/03_experiment_tracking/01_experiment_tracking/01_configuration.html) from `conf/<environment>/mlflow.yml`, which can be used to configure the experiment name:

- For **notebook** experiments, you have to configure the experiment name to match the full path of the notebook.
- For **workspace** experiments, the experiment name would be the full path to the experiments folder in the workspace.

To this end, let's add some OmegaConf syntax to `mlflow.yml` so that the experiment name can be specified from the outside, while taking a default value if not present:

In [0]:
%%writefile rocketfuel/conf/databricks/mlflow.yml
tracking:
  experiment:
    name: ${runtime_params:mlflow_experiment_name, ${kedro_root:}}

Let's try to set up a **notebook** experiment. For this, extract the notebook path:

In [0]:
notebook_path = dbutils.entry_point.getDbutils().notebook().getContext().notebookPath().get()
notebook_path

And pass that as a runtime parameter to specify the experiment name:

_Note: Extra params cannot contain spaces when passed to `%reload_kedro`, see [this issue](https://github.com/kedro-org/kedro/issues/4813)_

In [0]:
%reload_kedro rocketfuel --env databricks --params mlflow_experiment_name=$notebook_path

Now, every time a Kedro pipeline is run, it's logged as al MLflow run:

In [0]:
session.run("load_data")

![MLflow run corresponding to a Kedro run on Databricks](./kedro-databricks-mlflow-run.png)

### Log artifacts as MLflow models in the Unity Catalog

In [0]:
%sql
SHOW CATALOGS;

In [0]:
# Test code
from sklearn import datasets
from sklearn.ensemble import RandomForestClassifier

import mlflow

# Train a sklearn model on the iris dataset
X, y = datasets.load_iris(return_X_y=True, as_frame=True)
clf = RandomForestClassifier(max_depth=7)
clf.fit(X, y)

# Note that the UC model name follows the pattern
# <catalog_name>.<schema_name>.<model_name>, corresponding to
# the catalog, schema, and registered model name
# in Unity Catalog under which to create the version
# The registered model will be created if it doesn't already exist
autolog_run = mlflow.last_active_run()
model_uri = "runs:/{}/model".format(autolog_run.info.run_id)  # NOTE: Can this be automatic?
mlflow.register_model(model_uri, "aza-databricks-b9b7aae-catalog.rocketfuel.iris_model")  # NOTE: Can this be automatic?

In [0]:
session.run("__default__")

### Register models using the Databricks Unity Catalog

_**Note**: fsspec uses the DBFS API, which is not compatible with Unity Catalog according to https://github.com/fsspec/filesystem_spec/issues/1656_